

For the last few months The world is facing a serious threat named COVID-19. This microscopic object made made such an impact such that human existance is at  a huge risk. WHO declared this situation as a pandemic. 

Most people infected with the COVID-19 virus will experience mild to moderate respiratory illness and recover without requiring special treatment. Older people, and those with underlying medical problems like cardiovascular disease, diabetes, chronic respiratory disease, and cancer are more likely to develop serious illness.

The best way to prevent and slow down transmission is be well informed about the COVID-19 virus, the disease it causes and how it spreads. Protect yourself and others from infection by washing your hands or using an alcohol based rub frequently and not touching your face.

The COVID-19 virus spreads primarily through droplets of saliva or discharge from the nose when an infected person coughs or sneezes, so it’s important that you also practice respiratory etiquette (for example, by coughing into a flexed elbow).

At this time, there are no specific vaccines or treatments for COVID-19. However, there are many ongoing clinical trials evaluating potential treatments. WHO will continue to provide updated information as soon as clinical findings become available.

Stay informed:

Protect yourself: advice for the public Myth busters Questions and answers Situation reports All information on the COVID-19 outbreak


In [ ]:

#import required modules
import pandas as pd
import matplotlib as mat
import matplotlib.pyplot as plt
from PIL import Image
from wordcloud import WordCloud
import xgboost as xgb
from xgboost import plot_importance
from sklearn.model_selection import cross_val_score, GridSearchCV,cross_val_predict
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error,  r2_score
import numpy as np
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.express as px
import plotly.io as pio
import datetime


# Data Preprocessing

In [ ]:
mat.rcParams.update({'figure.figsize':(20,15),'font.size':14})
covid19_train = pd.read_csv('../input/covid19-global-forecasting-week-3/train.csv')
covid19_test = pd.read_csv('../input/covid19-global-forecasting-week-3/test.csv')
covid19_train.rename(columns={'Id':'ForecastId'},inplace=True)
data = pd.read_csv('../input/coviddata/time-series-19-covid-combined.csv')

In [ ]:
covid19_train['Date-All'] = covid19_train['Date'].str.replace('-','').astype(int)
covid19_test['Date-All'] = covid19_test['Date'].str.replace('-','').astype(int)
data['Date-All'] =data['Date'].str.replace('-','').astype(int)

covid19_train['Date'] = pd.to_datetime(covid19_train['Date'])
covid19_test['Date'] = pd.to_datetime(covid19_test['Date'])
data['Date'] = pd.to_datetime(data['Date'])

In [ ]:
covid19_gdf = covid19_train.groupby(['Date','Country_Region'])['ConfirmedCases'].sum().reset_index()
covid19_gdf['date'] = pd.to_datetime(covid19_gdf['Date'])
covid19_gdf['date'] = covid19_gdf['date'].dt.strftime('%m/%d/%Y')


data_gdf = data.groupby(['Date','Country/Region'])['Confirmed'].sum().reset_index()
data_gdf['date'] = pd.to_datetime(data['Date'])
data_gdf['date'] = data_gdf['date'].dt.strftime('%m/%d/%Y')



In [ ]:
d=data_gdf.drop("Date",axis=1)
d.tail(20).style.background_gradient(cmap="Reds")

# Data Visualization

In [ ]:
conf=pd.read_csv("../input/coviddata/time_series_covid19_confirmed_global.csv")
death=pd.read_csv("../input/coviddata/time_series_covid19_deaths_global.csv")
recov=pd.read_csv("../input/coviddata/time_series_covid19_recovered_global.csv")
cum=pd.read_csv("../input/coviddata/time_series_covid19_cumudf_global.csv")
daily=pd.read_csv("../input/coviddata/time_series_covid19_dailydf_global.csv")
cols=conf.keys()
confirmed = conf.loc[:, cols[4]:cols[-1]]
deaths = death.loc[:, cols[4]:cols[-1]]
recoveries = recov.loc[:, cols[4]:cols[-1]]


In [ ]:
#Confirmed Cases
data=pd.read_csv('../input/countrywisedata/countrywise data.csv')
#data.tail(4)

fig_c = px.choropleth(data, locations="Country", locationmode='country names', 
                      color=np.log(data["Confirmed"]), hover_name="Country", hover_data=['Confirmed'],title="Worldwide Confirmed Cases")

fig_c.show()


In [ ]:
#Death Cases
fig_c = px.choropleth(data, locations="Country", locationmode='country names', 
                      color=np.log(data["Deaths"]), hover_name="Country", hover_data=['Deaths'],title="Worldwide Fatalities ")

fig_c.show()

In [ ]:
fig = px.scatter_geo(covid19_gdf.fillna(0), locations="Country_Region", locationmode='country names', 
                     color="ConfirmedCases", size="ConfirmedCases", hover_name="Country_Region", 
                     projection="natural earth", animation_frame="date", 
                     title='COVID-19 Spread', color_continuous_scale="OrRd")
fig.update(layout_coloraxis_showscale=False)
fig.show()

In [ ]:
totalCountryCases = covid19_train.drop_duplicates(['Province_State', 'Country_Region'],keep='last').groupby(['Country_Region'])[['ConfirmedCases']].sum().sort_values('ConfirmedCases',ascending=False)
totalCountryCases.head(20).plot(kind='bar',color='r')
plt.xlabel("Country/Region")
plt.ylabel("Number of cases")
plt.grid()
plt.show()

In [ ]:
totalDailyCases = covid19_train.groupby(['Date'])[['ConfirmedCases']].sum().sort_values('ConfirmedCases',ascending=False)
totalDailyDeaths = covid19_train.groupby(['Date'])[['Fatalities']].sum().sort_values('Fatalities',ascending=False)


In [ ]:
totalDailyCases.plot(grid=False,marker='.',color='black',markersize=20)
plt.grid(color='r',linestyle='-')
plt.title("Confirmed Cases over the World")
totalDailyDeaths.plot(grid=False,marker='.',color='r',markersize=20)
plt.grid(color='r',linestyle='-')
plt.title("Fatality Cases over the World")
plt.show()


**Confirmed cases, fatalities and Recovery cases**

In [ ]:
col=cum.keys()
c=cum.loc[:,col[1]]
d=cum.loc[:,col[2]]
r=cum.loc[:,col[3]]
#make date column
start="01/22/2020"
start_date = datetime.datetime.strptime(start, '%m/%d/%Y')
date=[]
for i in range(len(cum["Days"])):
    date.append((start_date + datetime.timedelta(days=i)).strftime('%m/%d/%Y'))

#PLot
plt.figure(figsize=(16, 9))
plt.plot(cum['Days'],c,'b',label="Confirmed Cases",marker='.',markersize='15')
plt.plot(cum['Days'],d,'r',label="Deaths",marker='.',markersize='10')
plt.plot(cum['Days'],r,'g',label="Recovery",marker='.',markersize='12')
plt.grid()
plt.legend()
plt.show()


**Case studies for China, Italy, Spain, US and India**

In [ ]:
##case studies
china_cases = [] 
italy_cases = []
us_cases = [] 
spain_cases = [] 
india_cases=[]
days=cum['Days']
n=confirmed.keys()


In [ ]:
for i in n:
    china_cases.append(conf[conf['Country/Region']=='China'][i].sum())
    italy_cases.append(conf[conf['Country/Region']=='Italy'][i].sum())
    us_cases.append(conf[conf['Country/Region']=='US'][i].sum())
    spain_cases.append(conf[conf['Country/Region']=='Spain'][i].sum())
    india_cases.append(conf[conf["Country/Region"]=='India'][i].sum())


In [ ]:
plt.figure(figsize=(16, 9))
plt.plot(days, china_cases,'r',label="China",marker='.',markersize=13)
plt.plot(days, italy_cases,'g',label="Italy",marker='.',markersize=13)
plt.plot(days, us_cases,'b',label="US",marker='.',markersize=13)
plt.plot(days, spain_cases,'y',label="Spain",marker='.',markersize=13)
plt.title(' Confirmed Cases ', size=30)
plt.xlabel('Days', size=15)
plt.ylabel('Confirmed Cases', size=15)
plt.xticks(size=20)
plt.yticks(size=20)
plt.legend()
plt.show()

In [ ]:
#Death cases

china_death=[]
us_death=[]
spain_death=[]
italy_death=[]
india_death=[]

for i in n:
    china_death.append(death[death['Country/Region']=='China'][i].sum())
    italy_death.append(death[death['Country/Region']=='Italy'][i].sum())
    us_death.append(death[death['Country/Region']=='US'][i].sum())
    spain_death.append(death[death['Country/Region']=='Spain'][i].sum())
    india_death.append(death[death["Country/Region"]=='India'][i].sum())


In [ ]:
plt.figure(figsize=(16, 9))
plt.plot(days,china_death,'r',label="Death cases in China",marker=".",markersize='15')
plt.plot(days,italy_death,'b',label="Death cases in Italy",marker=".",markersize='15')
plt.plot(days,spain_death,'y',label="Death cases in Spain",marker=".",markersize='15')
plt.plot(days,us_death,'g',label="Death cases in US",marker=".",markersize='15')
plt.grid()
plt.xlabel("Days")
plt.ylabel("Death Cases")
plt.title("Fatalities ")
plt.legend()

plt.show()

In [ ]:
china_cases=np.array(china_cases).reshape(1,-1)
italy_cases=np.array(italy_cases).reshape(1,-1)
spain_cases=np.array(spain_cases).reshape(1,-1)
india_cases=np.array(india_cases).reshape(1,-1)
us_cases=np.array(us_cases).reshape(1,-1)


In [ ]:
#Mortality Rates
mort_china=china_death/china_cases
mort_italy=italy_death/italy_cases
mort_spain=spain_death/spain_cases
mort_india=india_death/india_cases
mort_us=us_death/us_cases

**Worldwide Mortality Rate**

In [ ]:
mortality_rate = []
recovery_rate = []
dates = confirmed.keys()
for i in dates:
    confirmed_sum = confirmed[i].sum()
    death_sum = deaths[i].sum()
    recovered_sum = recoveries[i].sum()
    
    # calculate rates
    mortality_rate.append(death_sum/confirmed_sum)
    recovery_rate.append(recovered_sum/confirmed_sum)

In [ ]:
#worldwide Mortality
mean_mortality_rate = np.mean(mortality_rate)
plt.figure(figsize=(16, 9))
plt.plot(days, mortality_rate, color='red')
plt.axhline(y = mean_mortality_rate,linestyle='--', color='black')
plt.title('Mortality Rate of Coronavirus Over Time', size=30)
plt.legend(['mortality rate', 'y='+str(mean_mortality_rate)], prop={'size': 20})
plt.xlabel('Days', size=20)
plt.ylabel('Mortality Rate', size=20)
plt.xticks(size=20)
plt.yticks(size=20)
plt.show()

**Worldwide Recovery Rate**

In [ ]:
recov_mean=np.mean(recovery_rate)
plt.figure(figsize=(16, 9))
plt.plot(days, recovery_rate, color='green')
plt.title(' Recovery rate Over Time', size=30)
plt.axhline(y=recov_mean,linestyle='--',color='black')
plt.legend(['recovery rate', 'y='+str(recov_mean)], prop={'size': 10})
plt.xlabel('Days', size=20)
plt.ylabel('Recovery Rate', size=20)
plt.xticks(size=20)
plt.yticks(size=20)
plt.show()


**Compare Mortality rates of China, Italy, Spain,US and India**

In [ ]:
#Visualization of Mortality Rates
plt.figure(figsize=(16, 9))
plt.plot(days,mort_china.transpose(),'r',label="Mortality of China")
plt.plot(days,mort_italy.transpose(),'b',label="Mortality of Italy")
plt.plot(days,mort_spain.transpose(),'y',label="Mortality of Spain")
plt.plot(days,mort_us.transpose(),'black',label="Mortality of US")
plt.plot(days,mort_india.transpose(),'orange',label="Mortality of India")
plt.legend()
#plt.xlabel('Days Since 1/22/2020', size=30)
plt.ylabel('Mortality', size=30)
plt.xticks(size=20)
plt.yticks(size=20)
plt.grid()
plt.show()

# S-I-R Model

# Forecast for week-3

Forecast using XGBOOST

In [ ]:
PATH_WEEK2='/kaggle/input/covid19-global-forecasting-week-3'
df_train = pd.read_csv(f'{PATH_WEEK2}/train.csv')
df_test = pd.read_csv(f'{PATH_WEEK2}/test.csv')
df_train.head()
df_test.head()
df_train.rename(columns={'Country_Region':'Country'}, inplace=True)
df_test.rename(columns={'Country_Region':'Country'}, inplace=True)

df_train.rename(columns={'Province_State':'State'}, inplace=True)
df_test.rename(columns={'Province_State':'State'}, inplace=True)

df_train['Date'] = pd.to_datetime(df_train['Date'], infer_datetime_format=True)
df_test['Date'] = pd.to_datetime(df_test['Date'], infer_datetime_format=True)

df_train.info()
df_test.info()

y1_Train = df_train.iloc[:, -2]
y1_Train.head()
y2_Train = df_train.iloc[:, -1]
y2_Train.head()

EMPTY_VAL = "EMPTY_VAL"

def fillState(state, country):
    if state == EMPTY_VAL: return country
    return state

In [ ]:
#X_Train = df_train.loc[:, ['State', 'Country', 'Date']]
X_Train = df_train.copy()

X_Train['State'].fillna(EMPTY_VAL, inplace=True)
X_Train['State'] = X_Train.loc[:, ['State', 'Country']].apply(lambda x : fillState(x['State'], x['Country']), axis=1)

X_Train.loc[:, 'Date'] = X_Train.Date.dt.strftime("%m%d")
X_Train["Date"]  = X_Train["Date"].astype(int)

X_Train.head()

#X_Test = df_test.loc[:, ['State', 'Country', 'Date']]
X_Test = df_test.copy()

X_Test['State'].fillna(EMPTY_VAL, inplace=True)
X_Test['State'] = X_Test.loc[:, ['State', 'Country']].apply(lambda x : fillState(x['State'], x['Country']), axis=1)

X_Test.loc[:, 'Date'] = X_Test.Date.dt.strftime("%m%d")
X_Test["Date"]  = X_Test["Date"].astype(int)

X_Test.head()

In [ ]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

X_Train.Country = le.fit_transform(X_Train.Country)
X_Train['State'] = le.fit_transform(X_Train['State'])

X_Train.head()

X_Test.Country = le.fit_transform(X_Test.Country)
X_Test['State'] = le.fit_transform(X_Test['State'])

X_Test.head()

df_train.head()
df_train.loc[df_train.Country == 'Afghanistan', :]
df_test.tail()

In [ ]:
from warnings import filterwarnings
filterwarnings('ignore')

from sklearn import preprocessing

le = preprocessing.LabelEncoder()

from xgboost import XGBRegressor

countries = X_Train.Country.unique()

#models_C = {}
#models_F = {}

df_out = pd.DataFrame({'ForecastId': [], 'ConfirmedCases': [], 'Fatalities': []})

for country in countries:
    states = X_Train.loc[X_Train.Country == country, :].State.unique()
    #print(country, states)
    # check whether string is nan or not
    for state in states:
        X_Train_CS = X_Train.loc[(X_Train.Country == country) & (X_Train.State == state), ['State', 'Country', 'Date', 'ConfirmedCases', 'Fatalities']]
        
        y1_Train_CS = X_Train_CS.loc[:, 'ConfirmedCases']
        y2_Train_CS = X_Train_CS.loc[:, 'Fatalities']
        
        X_Train_CS = X_Train_CS.loc[:, ['State', 'Country', 'Date']]
        
        X_Train_CS.Country = le.fit_transform(X_Train_CS.Country)
        X_Train_CS['State'] = le.fit_transform(X_Train_CS['State'])
        
        X_Test_CS = X_Test.loc[(X_Test.Country == country) & (X_Test.State == state), ['State', 'Country', 'Date', 'ForecastId']]
        
        X_Test_CS_Id = X_Test_CS.loc[:, 'ForecastId']
        X_Test_CS = X_Test_CS.loc[:, ['State', 'Country', 'Date']]
        
        X_Test_CS.Country = le.fit_transform(X_Test_CS.Country)
        X_Test_CS['State'] = le.fit_transform(X_Test_CS['State'])
        
        #models_C[country] = gridSearchCV(model, X_Train_CS, y1_Train_CS, param_grid, 10, 'neg_mean_squared_error')
        #models_F[country] = gridSearchCV(model, X_Train_CS, y2_Train_CS, param_grid, 10, 'neg_mean_squared_error')
        
        model1 = XGBRegressor(n_estimators=1000)
        model1.fit(X_Train_CS, y1_Train_CS)
        y1_pred = model1.predict(X_Test_CS)
        
        model2 = XGBRegressor(n_estimators=1000)
        model2.fit(X_Train_CS, y2_Train_CS)
        y2_pred = model2.predict(X_Test_CS)
        
        df = pd.DataFrame({'ForecastId': X_Test_CS_Id, 'ConfirmedCases': y1_pred, 'Fatalities': y2_pred})
        df_out = pd.concat([df_out, df], axis=0)
    # Done for state loop
# Done for country Loop

In [ ]:
df_out.ForecastId = df_out.ForecastId.astype('int')
df_out.tail()
#df_out.to_csv('submission.csv', index=False)

In [ ]:
Forecasting using ARIMA

In [ ]:
#Arima forecating
import warnings
warnings.filterwarnings(action='ignore')

import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima_model import ARIMA

def RMSLE(pred,actual):
    return np.sqrt(np.mean(np.power((np.log(pred+1)-np.log(actual+1)),2)))
pd.set_option('mode.chained_assignment', None)
test = pd.read_csv("../input/covid19-global-forecasting-week-3/test.csv")
train = pd.read_csv("../input/covid19-global-forecasting-week-3/train.csv")
train['Province_State'].fillna('', inplace=True)
test['Province_State'].fillna('', inplace=True)
train['Date'] =  pd.to_datetime(train['Date'])
test['Date'] =  pd.to_datetime(test['Date'])
train = train.sort_values(['Country_Region','Province_State','Date'])
test = test.sort_values(['Country_Region','Province_State','Date'])


feature_day = [1,20,50,100,200,500,1000]
def CreateInput(data):
    feature = []
    for day in feature_day:
        #Get information in train data
        data.loc[:,'Number day from ' + str(day) + ' case'] = 0
        if (train[(train['Country_Region'] == country) & (train['Province_State'] == province) & (train['ConfirmedCases'] < day)]['Date'].count() > 0):
            fromday = train[(train['Country_Region'] == country) & (train['Province_State'] == province) & (train['ConfirmedCases'] < day)]['Date'].max()        
        else:
            fromday = train[(train['Country_Region'] == country) & (train['Province_State'] == province)]['Date'].min()       
        for i in range(0, len(data)):
            if (data['Date'].iloc[i] > fromday):
                day_denta = data['Date'].iloc[i] - fromday
                data['Number day from ' + str(day) + ' case'].iloc[i] = day_denta.days 
        feature = feature + ['Number day from ' + str(day) + ' case']
    
    return data[feature]


pred_data_all = pd.DataFrame()
for country in train['Country_Region'].unique():
#for country in ['Vietnam']:
    for province in train[(train['Country_Region'] == country)]['Province_State'].unique():
        df_train = train[(train['Country_Region'] == country) & (train['Province_State'] == province)]
        df_test = test[(test['Country_Region'] == country) & (test['Province_State'] == province)]
        X_train = CreateInput(df_train)
        y_train_confirmed = df_train['ConfirmedCases'].ravel()
        y_train_fatalities = df_train['Fatalities'].ravel()
        X_pred = CreateInput(df_test)
        
        print(X_train)
        
        # Only train above 50 cases
        for day in sorted(feature_day,reverse = True):
            feature_use = 'Number day from ' + str(day) + ' case'
            idx = X_train[X_train[feature_use] == 0].shape[0]     
            if (X_train[X_train[feature_use] > 0].shape[0] >= 20):
                break
                                           
        adjusted_X_train = X_train[idx:][feature_use].values.reshape(-1, 1)
        adjusted_y_train_confirmed = y_train_confirmed[idx:]
        adjusted_y_train_fatalities = y_train_fatalities[idx:] #.values.reshape(-1, 1)
        idx = X_pred[X_pred[feature_use] == 0].shape[0]    
        adjusted_X_pred = X_pred[idx:][feature_use].values.reshape(-1, 1)
        
        pred_data = test[(test['Country_Region'] == country) & (test['Province_State'] == province)]
        max_train_date = train[(train['Country_Region'] == country) & (train['Province_State'] == province)]['Date'].max()
        min_test_date = pred_data['Date'].min()
        model = SARIMAX(adjusted_y_train_confirmed, order=(1,1,0), 
                        #seasonal_order=(1,1,0,12),
                        measurement_error=True).fit(disp=False)
        y_hat_confirmed = model.forecast(pred_data[pred_data['Date'] > max_train_date].shape[0])
        y_train_confirmed = train[(train['Country_Region'] == country) & (train['Province_State'] == province) & (train['Date'] >=  min_test_date)]['ConfirmedCases'].values
        y_hat_confirmed = np.concatenate((y_train_confirmed,y_hat_confirmed), axis = 0)
               
        model = SARIMAX(adjusted_y_train_fatalities, order=(1,1,0), 
                        #seasonal_order=(1,1,0,12),
                        measurement_error=True).fit(disp=False)
        y_hat_fatalities = model.forecast(pred_data[pred_data['Date'] > max_train_date].shape[0])
        y_train_fatalities = train[(train['Country_Region'] == country) & (train['Province_State'] == province) & (train['Date'] >=  min_test_date)]['Fatalities'].values
        y_hat_fatalities = np.concatenate((y_train_fatalities,y_hat_fatalities), axis = 0)
        
        
        pred_data['ConfirmedCases_hat'] =  y_hat_confirmed
        pred_data['Fatalities_hat'] = y_hat_fatalities
        pred_data_all = pred_data_all.append(pred_data)

df_val = pd.merge(pred_data_all,train[['Date','Country_Region','Province_State','ConfirmedCases','Fatalities']],on=['Date','Country_Region','Province_State'], how='left')
df_val.loc[df_val['Fatalities_hat'] < 0,'Fatalities_hat'] = 0
df_val.loc[df_val['ConfirmedCases_hat'] < 0,'ConfirmedCases_hat'] = 0
df_val_3 = df_val.copy()


In [ ]:
#Submission csv
submission = df_val[['ForecastId','ConfirmedCases_hat','Fatalities_hat']]
submission.columns = ['ForecastId','ConfirmedCases','Fatalities']
submission.to_csv('submission.csv', index=False)
